# run-zurich-validation.sh

Inside the `run-zurich-validation.sh` script, we have a few different test scenarios that we want to run:`validate-data`, `validate-machine-status`, and `validate-total-counter`. In this notebook we will be able to run through each of these tests and see the individualized results. 

For the purposes of this notebook, we will skip this cell and run the tests individually.

In [ ]:
# to run full integration tests
./run-zurich-validation.sh

./validations/validate-data.sh: line 3: ../../utils/set-colors.sh: No such file or directory
 => Running input watcher on zurich/input 
Setting up watches.
Watches established.
 => Running output watcher on metrics/dotnet/machine-status 
 => Running output watcher on zurich/input/valid 
 => Running output watcher on metrics/machine-status 
 => Running output watcher on zurich/input/invalid 
 => Running output watcher on metrics/dotnet/total-count 
 => Running output watcher on zurich/debug 
 => Running output watcher on metrics/total-count 
Processing ./assets/zurich-validation/data-validation/invalid/invalid-empty-payload.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-01.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-02.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-missing-messageType.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-type-messageTyp

## Running the Tests
Change the notebook environment to the `test/integration` directory.

In [5]:
cd /workspaces/aio-evaluation/test/integration/

This first section sets up the test environment and creates the watchers needed to run the tests. 

Once this is run you should see a new directory called `.workspace` which contains the `logs`, `metrics`, and `zurich` directories with the various outputs in json files. The `logs` directory contains the logs for the tests, the `metrics` directory contains the machine status and total counter topic outputs, and the `zurich` directory contains the input and processed input json files for the tests.

In [6]:
#!/bin/bash

. ../utils/set-colors.sh
. ../utils/create-workspace.sh
. ./data-feeder/run-input-watchers.sh
. ./data-feeder/run-output-watchers.sh
. ./validations/validate-data.sh
. ./validations/validate-machine-status.sh
. ./validations/validate-total-counter.sh

declare -A topics_and_logs
topics_and_logs=(
  ["zurich/input/valid"]="zurich-valid.log"
  ["zurich/input/invalid"]="zurich-invalid.log"
  ["zurich/debug"]="zurich-debug.log"
  ["metrics/aio/machine-status"]="zurich-aio-machine-status.log"
  ["metrics/aio/total-count"]="zurich-aio-total-count.log"
)

WORKSPACE=".workspace"
INPUT_TOPIC="zurich/input"
AIO_METRICS_TOPIC="metrics/aio"

create_workspace $WORKSPACE
run_input_watcher $INPUT_TOPIC
run_output_watchers

 => Running input watcher on zurich/input 
[7] 275719
Setting up watches.
Watches established.
[7] 275719
Setting up watches.
Watches established.
 => Running output watcher on metrics/aio/total-count 
[8] 275734
 => Running output watcher on zurich/input/valid 
[9] 275735
 => Running output watcher on metrics/aio/machine-status 
[10] 275736
 => Running output watcher on zurich/input/invalid 
[11] 275737
 => Running output watcher on zurich/debug 
[12] 275738


Next we will run the `validate-data` test. This will iterate over the input files in the `/assets/zurich-validation/data-validation` directory and validate the data. We have 2 types of inputs: `valid-*.json` and `invalid-*.json`. The `valid-*.json` files are correct input payloads and should pass the validation. The `invalid-*.json` files should fail the validation based on various issues with the payload.

In [4]:
# test for valid input data
test_for_valid_data

Processing ./assets/zurich-validation/data-validation/invalid/invalid-empty-payload.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-01.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-02.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-01.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-02.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-missing-messageType.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-type-messageType.json
Processing ./assets/zurich-validation/data-validation/valid/valid-01.json
Processing ./assets/zurich-validation/data-validation/valid/valid-02.json
Processing ./assets/zurich-validation/data-validation/valid/valid-03.json
 => [EXPECTED] Valid: 3, Invalid: 5 
 => Waiting for processing [8 seconds] ... 

 => [TEST RESULTS] 

 INFO: 3 messages found 
 INFO:

The next test is the `total-counter` test. This test will iterate over the input files in the `/assets/zurich-validation/total-counter` directory and validate the data. It will pass in various payloads to the total-counter pipeline and check the output to make sure it is correct.

In [2]:
# test for total counter scenarios
# Scenario 1: total count = 1
# Scenario 4: total count = 250 with multiple shifts

test_for_total_counter "$AIO_METRICS_TOPIC"

Processing ./assets/zurich-validation/start-shift-total-count/start-shift-good-counter-mill.json
Processing ./assets/zurich-validation/start-shift-total-count/start-shift-defect-counter-mill.json
Processing ./assets/zurich-validation/total-counter/total-count-1/good-counter-1-mill.json
 Current AIO count: 1770 (from: .workspace/metrics/aio/total-count) 
Processing ./assets/zurich-validation/start-shift-total-count/start-shift-good-counter-mill.json
Processing ./assets/zurich-validation/start-shift-total-count/start-shift-defect-counter-mill.json
Processing ./assets/zurich-validation/total-counter/total-count-150-mill/0-defect-counter-60-mill.json
Processing ./assets/zurich-validation/total-counter/total-count-150-mill/1-defect-counter-100-oven.json
Processing ./assets/zurich-validation/total-counter/total-count-150-mill/2-good-counter-90-mill.json
 INFO: [.workspace/metrics/aio/total-count] 20240301220051074.json contains expected value of 1920. 
Processing ./assets/zurich-validation/s

Finally, we will test through the various machine status scenarios. This test will iterate over the input files in the `/assets/zurich-validation/machine-status` directory and validate the data. It will pass in various payloads to the machine-status pipeline and check the output to make sure it is correct. 

In [7]:
# test for machine status scenarios
# Scenario 1: machine status = "fault" or 0
# Scenario 2: machine status = "automatic" or 1
# Scenario 3: machine status = "idle" or 3
# Scenario 4: machine status = "manual" or 2
# Scenario 5: machine status = "maintenance" or 99

test_for_machine_status "$AIO_METRICS_TOPIC"

Processing ./assets/zurich-validation/machine-status/Fault/Fault-true.json
Processing ./assets/zurich-validation/machine-status/Fault/Fault-false.json
 INFO: [.workspace/metrics/aio/machine-status] 20240301222325485.json contains expected value of 0. 
Processing ./assets/zurich-validation/machine-status/automatic/Fault-false.json
Processing ./assets/zurich-validation/machine-status/automatic/automatic-true.json
Processing ./assets/zurich-validation/machine-status/automatic/Fault-false.json
Processing ./assets/zurich-validation/machine-status/automatic/idle-false.json
Processing ./assets/zurich-validation/machine-status/automatic/manual-false.json
 INFO: [.workspace/metrics/aio/machine-status] 20240301222330554.json contains expected value of 1. 
Processing ./assets/zurich-validation/machine-status/Fault/Fault-true.json
 INFO: [.workspace/metrics/aio/machine-status] 20240301222331593.json contains expected value of 0. 
Processing ./assets/zurich-validation/machine-status/idle/Fault-fals

In order to rerun tests, you will need to stop background processes created from previous tests.

In [6]:
/workspaces/aio-evaluation/test/integration/clean-integration-tests.sh

pkill: killing pid 38855 failed: Operation not permitted
pkill: killing pid 38937 failed: Operation not permitted
pkill: killing pid 40605 failed: Operation not permitted
pkill: killing pid 40827 failed: Operation not permitted
pkill: killing pid 179398 failed: Operation not permitted
pkill: killing pid 179399 failed: Operation not permitted
pkill: killing pid 179443 failed: Operation not permitted
pkill: killing pid 179446 failed: Operation not permitted


: 1